##Import Modules

In [1]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

##Import S&P tickers

In [3]:
#download sp500 data from gdrive
!gdown 1BObMwwO42SnD08qXn33m_8PZ3Ssril-K

Downloading...
From: https://drive.google.com/uc?id=1BObMwwO42SnD08qXn33m_8PZ3Ssril-K
To: /content/sp500_companies.csv
100% 802k/802k [00:00<00:00, 114MB/s]


In [4]:
stocks = pd.read_csv('sp500_companies.csv')
stocks = stocks.loc[:,['Symbol']]
stocks.columns = ['Ticker']
print(stocks)

    Ticker
0     AAPL
1     MSFT
2    GOOGL
3     TSLA
4    BRK.B
..     ...
488   PENN
489    ALK
490    VNO
491   NCLH
492    PVH

[493 rows x 1 columns]


##Acquiring API Token

In [5]:
#download API Token from gdrive
!gdown 1UU-1Lu1PEzE895LXxv1cZMKoziKNfJN1

Downloading...
From: https://drive.google.com/uc?id=1UU-1Lu1PEzE895LXxv1cZMKoziKNfJN1
To: /content/secrets.py
100% 60.0/60.0 [00:00<00:00, 97.2kB/s]


In [6]:
from secrets import IEX_CLOUD_API_TOKEN
token = IEX_CLOUD_API_TOKEN

##Looping through stocks with Batch API Calls
###Saving Ticker, Stock Price & Market Cap

In [7]:
def chunks(lst, n):
  for i in range(0, len(lst), n):
    yield lst[i:i+n]

In [8]:
my_columns = ['Ticker', 'Stock Price', 'Market Cap', 'Weigths','Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Cap,Weigths,Number of Shares to Buy


In [9]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
token = IEX_CLOUD_API_TOKEN
for i in range(0, len(symbol_groups)):
  symbol_strings.append(','. join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
  api_url = 'https://sandbox.iexapis.com/stable'
  batch_api_call_url = api_url + f'/stock/market/batch?symbols={symbol_string}&types=quote&token={token}'
  data = requests.get(batch_api_call_url).json()
  for symbol in symbol_string.split(','):
    final_dataframe = final_dataframe.append(
      pd.Series(
      [
        symbol,
        data[symbol]['quote']['latestPrice'],
        data[symbol]['quote']['marketCap'],
        'N/A',
        'N/A'
      ],
      index = my_columns
      ),
    ignore_index=True
    )
final_dataframe

,Ticker,Stock Price,Market Cap,Weigths,Number of Shares to Buy
0,AAPL,181.26,2904497822193,N/A,N/A
1,MSFT,303.13,2189660479075,N/A,N/A
2,GOOGL,120.30,84412944151,N/A,N/A
3,TSLA,943.31,996159532440,N/A,N/A
4,BRK.B,313.99,698563506396,N/A,N/A
...,...,...,...,...,...
488,PENN,36.54,5783911331,N/A,N/A
489,ALK,46.77,6176362327,N/A,N/A
490,VNO,29.86,5829210140,N/A,N/A
491,NCLH,13.96,3053325331,N/A,N/A


##Number of shares to buy

In [10]:
porfolio_size = input('Enter the value of your portfolio: ')

try:
  val = float(porfolio_size)
except ValueError:
  print('Please enter an integer')
  porfolio_size = input('Enter the value of your portfolio: ')
  val = float(porfolio_size)

Enter the value of your portfolio: 1000000


In [11]:
#save weights
total_mcap = final_dataframe['Market Cap'].sum()
for i in range(0, len(final_dataframe.index)):
  final_dataframe.loc[i,'Weigths'] = final_dataframe.loc[i, 'Market Cap']*100/total_mcap
final_dataframe

,Ticker,Stock Price,Market Cap,Weigths,Number of Shares to Buy
0,AAPL,181.26,2904497822193,7.939818,N/A
1,MSFT,303.13,2189660479075,5.985718,N/A
2,GOOGL,120.30,84412944151,0.230754,N/A
3,TSLA,943.31,996159532440,2.72313,N/A
4,BRK.B,313.99,698563506396,1.909613,N/A
...,...,...,...,...,...
488,PENN,36.54,5783911331,0.015811,N/A
489,ALK,46.77,6176362327,0.016884,N/A
490,VNO,29.86,5829210140,0.015935,N/A
491,NCLH,13.96,3053325331,0.008347,N/A


In [12]:
#Compute number of shares to buy
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
  final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor((val*(final_dataframe.loc[i, 'Weigths']/100))/final_dataframe.loc[i, 'Stock Price'])

final_dataframe.sort_values('Market Cap', ascending=False)

,Ticker,Stock Price,Market Cap,Weigths,Number of Shares to Buy
0,AAPL,181.26,2904497822193,7.939818,438
1,MSFT,303.13,2189660479075,5.985718,197
109,AMZN,147.24,1475447037331,4.033324,273
3,TSLA,943.31,996159532440,2.72313,28
4,BRK.B,313.99,698563506396,1.909613,60
...,...,...,...,...,...
487,DXC,27.32,6058821794,0.016563,6
490,VNO,29.86,5829210140,0.015935,5
488,PENN,36.54,5783911331,0.015811,4
492,PVH,71.40,4819010690,0.013173,1


##Save to excel

In [13]:
#initializing writer
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.sort_values('Market Cap', ascending=False).to_excel(writer, 'Recommended Trades', index = False)

In [14]:
#format
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_format = writer.book.add_format(
        {
            'num_format':'0.000000',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [15]:
#write
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Weigths', float_format],
                    'E': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [16]:
#save
writer.save()